In [188]:
import pandas as pd

In [189]:
#Visualization
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns
from pandas.tools.plotting import scatter_matrix

#Configure Visualization Defaults
# %matplotlib inline = show plots in Jupyter Notebook browser
%matplotlib inline
mpl.style.use('ggplot')
sns.set_style('white')
pylab.rcParams['figure.figsize'] = 12,8

In [190]:
from sklearn import ensemble, tree

from sklearn import metrics
from sklearn import model_selection
from sklearn import feature_selection
from sklearn.metrics import mean_absolute_error

In [256]:
# data 
melb_data = pd.read_csv("./data/melb_data.csv")
data = melb_data.copy()

In [257]:
# missing values
missing_cols = [col for col in data.columns 
                                 if data[col].isnull().any()]
print(missing_cols)

['Car', 'BuildingArea', 'YearBuilt', 'CouncilArea']


In [193]:
print(data.isnull().sum())

Suburb              0
Address             0
Rooms               0
Type                0
Price               0
Method              0
SellerG             0
Date                0
Distance            0
Postcode            0
Bedroom2            0
Bathroom            0
Car                62
Landsize            0
BuildingArea     6450
YearBuilt        5375
CouncilArea      1369
Lattitude           0
Longtitude          0
Regionname          0
Propertycount       0
dtype: int64


In [258]:
council_area_value_counts = data['CouncilArea'].value_counts()
# print(council_area_value_counts)
#df[df['Value']==df['Value'].max()]
most_common_council_area = council_area_value_counts.keys()[0]
# print(most_common_council_area)
data['CouncilArea'] = data['CouncilArea'].fillna(most_common_council_area)
missing_cols.remove('CouncilArea')

In [259]:
print(data['CouncilArea'].unique().tolist())

['Yarra', 'Moonee Valley', 'Port Phillip', 'Darebin', 'Hobsons Bay', 'Stonnington', 'Boroondara', 'Monash', 'Glen Eira', 'Whitehorse', 'Maribyrnong', 'Bayside', 'Moreland', 'Manningham', 'Banyule', 'Melbourne', 'Kingston', 'Brimbank', 'Hume', 'Knox', 'Maroondah', 'Casey', 'Melton', 'Greater Dandenong', 'Nillumbik', 'Whittlesea', 'Frankston', 'Macedon Ranges', 'Yarra Ranges', 'Wyndham', 'Cardinia', 'Unavailable', 'Moorabool']


In [260]:
# print(data['Car'].unique().tolist())
# data['Car'] = data['Car'].fillna(data['Car'].median())
# # print(data['Car'].median())
# print(data['Car'].unique().tolist())
for m_col in missing_cols:
    data[m_col] = data[m_col].fillna(data[m_col].median())

In [203]:
 print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 21 columns):
Suburb           13580 non-null object
Address          13580 non-null object
Rooms            13580 non-null int64
Type             13580 non-null object
Price            13580 non-null float64
Method           13580 non-null object
SellerG          13580 non-null object
Date             13580 non-null object
Distance         13580 non-null float64
Postcode         13580 non-null float64
Bedroom2         13580 non-null float64
Bathroom         13580 non-null float64
Car              13580 non-null float64
Landsize         13580 non-null float64
BuildingArea     13580 non-null float64
YearBuilt        13580 non-null float64
CouncilArea      13580 non-null object
Lattitude        13580 non-null float64
Longtitude       13580 non-null float64
Regionname       13580 non-null object
Propertycount    13580 non-null float64
dtypes: float64(12), int64(1), object(8)
memory usage: 2.2+ M

In [261]:
data = pd.concat([data, pd.get_dummies(data['Method'])], axis=1)
data = pd.concat([data, pd.get_dummies(data['Type'])], axis=1)
data = pd.concat([data, pd.get_dummies(data['Regionname'])], axis=1)
data = pd.concat([data, pd.get_dummies(data['SellerG'])], axis=1)
data = pd.concat([data, pd.get_dummies(data['Suburb'])], axis=1)
data = pd.concat([data, pd.get_dummies(data['CouncilArea'])], axis=1)

In [262]:
print(data[['Date', 'Address']].head())


        Date           Address
0  3/12/2016      85 Turner St
1  4/02/2016   25 Bloomburg St
2  4/03/2017      5 Charles St
3  4/03/2017  40 Federation La
4  4/06/2016       55a Park St


In [263]:
address_street_name = []
data['Address_List'] = data['Address'].map(lambda x: x.split(' '))
data['Address_Number'] = data['Address_List'].map(lambda x: x[0])
data['Address_Name'] = data['Address_List'].map(lambda x: x[1])
data['Address_Street'] = data['Address_List'].map(lambda x: x[2])
# print(len(data['Address'].unique().sum()))

In [264]:
data = pd.concat([data, pd.get_dummies(data['Address_Street'])], axis=1)
address_street = data['Address_Street'].unique().tolist()

data = pd.concat([data, pd.get_dummies(data['Address_Name'])], axis=1)
address_name = data['Address_Name'].unique().tolist()

In [239]:
print(address_name)

['Turner', 'Bloomburg', 'Charles', 'Federation', 'Park', 'Yarra', 'Nicholson', 'Valiant', 'Grosvenor', 'William', 'Abbotsford', 'Henry', 'Gipps', 'Stafford', 'Studley', 'Halsey', 'Bedford', 'Hart', 'Hawker', 'Hillside', 'McIntosh', 'Marshall', 'Roberts', 'Harrington', 'Parer', 'Clydesdale', 'Deidre', 'Etzel', 'Cameron', 'Victory', 'Earl', 'King', 'Kittyhawk', 'Fullarton', 'South', 'North', 'Hilbert', 'Moorna', 'Kerferd', 'Richardson', 'Beaconsfield', 'Graham', 'Montague', 'Brooke', 'Mills', 'Moubray', 'Faussett', 'Herbert', 'Draper', 'Dundas', 'Finlay', 'Gatehouse', 'Barrett', 'Page', 'Withers', 'Merton', 'Greig', 'Danks', 'Reed', 'Bridport', 'Durham', 'Smith', 'Yarralea', 'Fulham', 'Shiers', 'Harker', 'Miller', 'Yarana', 'Heidelberg', 'Naroon', 'Grange', 'Bennett', 'Hamilton', 'Chandler', 'Lowther', 'Margaret', 'Queen', 'Bracken', 'Blyth', 'Linnet', 'Rose', 'Grieve', 'Maidstone', 'Keeshan', 'David', 'Merritt', 'Millers', 'Pier', 'Wren', 'Rayner', 'Sargood', 'Fenfield', 'Seves', 'McInt

In [103]:
# suburb_treshold = 100
# below_suburb_treshold = []
# above_suburb_treshold = []
# suburb_counts = data['Suburb'].value_counts()
# # print(suburb_counts.describe())
# # print(suburb_counts['Reservoir'])
# for s_count in suburb_counts.iteritems():
#     if s_count[1] < suburb_treshold:
#         below_suburb_treshold.append(s_count[0])
# # print(below_suburb_treshold)
# data['Suburb_Misc'] = data['Suburb'].map(lambda x: 1 if x in below_suburb_treshold else 0)

# for suburb in data['Suburb'].unique().tolist():
#     if suburb not in below_suburb_treshold: 
#         above_suburb_treshold.append(suburb)
#         data[suburb] = data['Suburb'].map(lambda x: 1 if x == suburb else 0)

In [96]:
    # data = pd.concat([data, pd.get_dummies(data['SellerG'])], axis=1)
    # seller_g_treshold = 100
    # below_seller_g_treshold = []
    # above_seller_g_treshold = []
    # seller_g_counts = data['SellerG'].value_counts()
    # for sg_count in seller_g_counts.iteritems():
    #     if sg_count[1] < seller_g_treshold:
    #         below_seller_g_treshold.append(sg_count[0])
    # # print(below_suburb_treshold)
    # data['SellerG_Misc'] = data['SellerG'].map(lambda x: 1 if x in below_seller_g_treshold else 0)

    # for sg in data['SellerG'].unique().tolist():
    #     if sg not in below_seller_g_treshold: 
    #         above_seller_g_treshold.append(sg)
    #         data[sg] = data['SellerG'].map(lambda x: 1 if x == sg else 0)

In [265]:
data['Type'].head()

,Type,Type
0,h,0
1,h,0
2,h,0
3,h,0
4,h,0


In [254]:
features = data['SellerG'].unique().tolist() \
+ ['Car', 
   'YearBuilt', 'Rooms', 'BuildingArea'
   'Distance', 
   'Postcode', 
   'Bathroom', 'Bedroom2', 'Propertycount', 'Longtitude', 'Lattitude', 'Landsize'] \
+ address_name \
+ data['Suburb'].unique().tolist() \
+ data['Method'].unique().tolist() \
# + data['Type'].unique().tolist() \
# + data['Regionname'].unique().tolist() \
# + data['CouncilArea'].unique().tolist()
target = ['Price']

In [232]:
train_X, test_X, train_y, test_y = model_selection.train_test_split(data[features], data[target], random_state=0)

In [233]:
# define models
decision_tree_model = tree.DecisionTreeRegressor(max_depth=11)
random_forest_model = ensemble.RandomForestRegressor()

In [234]:
# fit models
decision_tree_model.fit(train_X, train_y['Price'].tolist())
random_forest_model.fit(train_X,  train_y['Price'].tolist())

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

In [235]:
# compare socores
decision_tree_train_acc = decision_tree_model.score(train_X, train_y['Price'].tolist())*100
decision_tree_test_acc = decision_tree_model.score(test_X, test_y['Price'].tolist())*100
random_forest_train_acc = random_forest_model.score(train_X, train_y['Price'].tolist())*100
random_forest_test_acc = random_forest_model.score(test_X, test_y['Price'].tolist())*100
print("train accuracy: (decision tree): {:.3f}% (random forest tree) {:.3f}%".format(decision_tree_train_acc, random_forest_train_acc))
print("test accuracy: (decision tree): {:.3f}% (random forest tree) {:.3f}%".format(decision_tree_test_acc, random_forest_test_acc))

train accuracy: (decision tree): 90.869% (random forest tree) 96.470%
test accuracy: (decision tree): 64.825% (random forest tree) 77.254%


In [220]:
max_acc_tree = 0
max_acc_rftree = 0
for max_depth in range(1, 19):
    for max_leaf_nodes in range(-10, 0):
        dt_mod = tree.DecisionTreeRegressor(max_depth=max_depth, max_leaf_nodes=max_leaf_nodes)
        rftree_mod = ensemble.RandomForestRegressor(max_depth=max_depth, max_leaf_nodes=max_leaf_nodes)
        tree_acc = dt_mod.fit(train_X, train_y['Price'].tolist()).score(test_X, test_y['Price'])
        rftree_acc = rftree_mod.fit(train_X, train_y['Price']).score(test_X, test_y['Price'])
        if max_acc_tree<tree_acc:
            max_acc_tree = tree_acc
            print("test accuracy: (decision tree): {:.3f}% (random forest tree) {:.3f}%".format(tree_acc*100, rftree_acc*100))  
        if max_acc_rftree<rftree_acc:
            max_acc_rftree = rftree_acc
            print("test accuracy: (decision tree): {:.3f}% (random forest tree) {:.3f}%".format(tree_acc*100, rftree_acc*100))  

test accuracy: (decision tree): 15.811% (random forest tree) 15.810%
test accuracy: (decision tree): 15.811% (random forest tree) 15.810%
test accuracy: (decision tree): 15.811% (random forest tree) 17.695%
test accuracy: (decision tree): 15.811% (random forest tree) 19.065%
test accuracy: (decision tree): 15.811% (random forest tree) 20.045%
test accuracy: (decision tree): 15.811% (random forest tree) 20.297%
test accuracy: (decision tree): 37.085% (random forest tree) 37.088%
test accuracy: (decision tree): 37.085% (random forest tree) 37.088%
test accuracy: (decision tree): 37.085% (random forest tree) 39.160%
test accuracy: (decision tree): 37.085% (random forest tree) 39.295%
test accuracy: (decision tree): 37.085% (random forest tree) 40.852%
test accuracy: (decision tree): 37.085% (random forest tree) 41.049%
test accuracy: (decision tree): 49.960% (random forest tree) 52.031%
test accuracy: (decision tree): 49.960% (random forest tree) 52.031%
test accuracy: (decision tree): 49